In [34]:

#core enviroment libraries for RL 
import gym
from gym.spaces import Discrete, MultiDiscrete,Box, Dict, MultiBinary,Tuple
#utilities 
import numpy as np
import random
#these libraries have to do with the agents 
import ray
from ray.rllib.utils.deprecation import Deprecated
from ray.rllib.utils.metrics import (
    LAST_TARGET_UPDATE_TS,
    NUM_AGENT_STEPS_SAMPLED,
    NUM_ENV_STEPS_SAMPLED,
    NUM_TARGET_UPDATES,
    SYNCH_WORKER_WEIGHTS_TIMER,
)
from ray.rllib.utils.replay_buffers.utils import sample_min_n_steps_from_buffer
from ray.rllib.utils.typing import ResultDict, AlgorithmConfigDict
from ray.rllib.utils.deprecation import DEPRECATED_VALUE
from ray.rllib.utils.deprecation import deprecation_warning
from ray.rllib.agents.ppo import PPOTrainer
from ray.rllib.env.multi_agent_env import MultiAgentEnv
from ray import air, tune

import argparse
import logging
import os
#import pathpy as pp

from ray.tune import register_env
from ray.rllib.algorithms.qmix import QMixConfig
from ray.rllib.env.multi_agent_env import ENV_STATE
from ray.rllib.examples.env.two_step_game import TwoStepGame
from ray.rllib.policy.policy import PolicySpec
from ray.rllib.utils.test_utils import check_learning_achieved

import mne# preprocessing and brain importation and utilities library including acessing and preprocessing the EEG data
#these libraries have to do with the free energy principle
#import pymdp
#from pymdp import utils
#from pymdp.agent import Agent
#from gym.spaces import 

#from ray.rllib.algorithms.qmix.qmix_policy import QMixTorchPolicy

#optimization of deep learning and RL aspects of algorithm these will allow the algorithm to run faster with less memory 
#from composer import Trainer
#from nebullvm.api.functions import optimize_model 
from numba import jit

"""
dependency network

Qmix.py - has qmixpolicy.py as a dependency 
Qmixpolicy.py has  mixers.py and Model.py dependencies
Model.py -base
mixers.py -base

"""


#from ray.rllib.utils.torch_utils import 
"""
"""

'\n'

In [2]:
import tensorflow as tf

import ivy# library for interoperable across all deep learning frameworks 
#import tensorflow_datasets as tfds
import tensorflow_probability as tfp

from laplace import Laplace #for model selection 
from laplace.baselaplace import FullLaplace
from laplace.curvature.backpack import BackPackGGN
#from nebulgym.decorators.torch_decorators import accel

In [3]:
#from nebulgym.decorators.torch_decorators import accelerate_model, accelerate_dataset

#below libraries are core libraries for q-mix Rllib algorithm
from ray.rllib.models.modelv2 import ModelV2
from ray.rllib.models.preprocessors import get_preprocessor
from ray.rllib.models.torch.torch_modelv2 import TorchModelV2
from ray.rllib.utils.annotations import override
from ray.rllib.utils.framework import try_import_torch
from torch import nn

from ray.rllib.policy.torch_policy import TorchPolicy

from typing import Optional, Type,  Dict, List, Tuple

from ray.rllib.algorithms.simple_q.simple_q import SimpleQ, SimpleQConfig
from ray.rllib.algorithms.qmix.qmix_policy import QMixTorchPolicy
from ray.rllib.utils.replay_buffers.utils import update_priorities_in_replay_buffer
from ray.rllib.execution.rollout_ops import (
    synchronous_parallel_sample,
)

In [4]:
from ray.rllib.execution.train_ops import (
    multi_gpu_train_one_step,
    train_one_step,
)
from ray.rllib.policy.policy import Policy
from ray.rllib.models.preprocessors import get_preprocessor

import logging
import tree  # pip install dm_tree

from ray.rllib.algorithms.qmix.mixers import VDNMixer, QMixer
from ray.rllib.algorithms.qmix.model import RNNModel, _get_size
from ray.rllib.env.multi_agent_env import ENV_STATE
from ray.rllib.env.wrappers.group_agents_wrapper import GROUP_REWARDS
from ray.rllib.models.catalog import ModelCatalog
from ray.rllib.models.modelv2 import _unpack_obs
from ray.rllib.models.torch.torch_action_dist import TorchCategorical
from ray.rllib.policy.rnn_sequencing import chop_into_sequences
from ray.rllib.policy.sample_batch import SampleBatch
from ray.rllib.utils.framework import try_import_torch
from ray.rllib.utils.metrics.learner_info import LEARNER_STATS_KEY
from ray.rllib.utils.typing import TensorType

In [5]:
import nitime
from deeptime.sindy import SINDy

In [ ]:
from numpy.random import default_rng
rng = default_rng(seed=111)
#causal inference libraries
from timeawarepc.tpc import cfc_tpc, cfc_pc, cfc_gc
#from timeawarepc.simulate_data import *
#from timeawarepc.find_cfc import *

In [1]:
print("asdf")

asdf


In [6]:
class RNNModel(TorchModelV2, nn.Module,ivy.Module):
    """The default RNN model for QMIX."""

    def __init__(self, obs_space, action_space, num_outputs, model_config, name):
        TorchModelV2.__init__(
            self, obs_space, action_space, num_outputs, model_config, name
        )
        nn.Module.__init__(self)
        self.obs_size = _get_size(obs_space)
        self.rnn_hidden_dim = model_config["lstm_cell_size"]
        self.fc1 = nn.Linear(self.obs_size, self.rnn_hidden_dim)
        self.rnn = nn.GRUCell(self.rnn_hidden_dim, self.rnn_hidden_dim)
        self.fc2 = nn.Linear(self.rnn_hidden_dim, num_outputs)
        self.n_agents = model_config["n_agents"]

    @override(ModelV2)
    def get_initial_state(self):
        # Place hidden states on same device as model.
        return [
            self.fc1.weight.new(self.n_agents, self.rnn_hidden_dim).zero_().squeeze(0)
        ]
    #accelerate_model()# this will accelerate the model 
    @override(ModelV2)
    def forward(self, input_dict, hidden_state, seq_lens):
        x = nn.functional.relu(self.fc1(input_dict["obs_flat"].float()))
        h_in = hidden_state[0].reshape(-1, self.rnn_hidden_dim)
        h = self.rnn(x, h_in)
        q = self.fc2(h)
        return q, [h]


def _get_size(obs_space):
    return get_preprocessor(obs_space)(obs_space).size
#model = RNNModel()

In [7]:
import torch

In [10]:
from ray.rllib.models.modelv2 import ModelV2
from ray.rllib.models.preprocessors import get_preprocessor
from ray.rllib.models.torch.torch_modelv2 import TorchModelV2
from ray.rllib.utils.annotations import override
from ray.rllib.utils.framework import try_import_torch

torch, nn = try_import_torch()


class bmodel(TorchModelV2, nn.Module,ivy.Module):
    """The default RNN model for QMIX."""

    def __init__(self, obs_space, action_space, num_outputs, model_config, name):
        TorchModelV2.__init__(
            self, obs_space, action_space, num_outputs, model_config, name
        )
        nn.Module.__init__(self)
        self.obs_size = _get_size(obs_space)
        self.rnn_hidden_dim = model_config["lstm_cell_size"]
        self.fc1 = nn.Linear(self.obs_size, self.rnn_hidden_dim)
        self.rnn = nn.GRUCell(self.rnn_hidden_dim, self.rnn_hidden_dim)
        self.fc2 = nn.Linear(self.rnn_hidden_dim, num_outputs)
        self.n_agents = model_config["n_agents"]

    @override(ModelV2)
    def get_initial_state(self):
        # Place hidden states on same device as model.
        return [
            self.fc1.weight.new(self.n_agents, self.rnn_hidden_dim).zero_().squeeze(0)
        ]

    #@override(ModelV2)
    def _forward(self,x,input_dict, hidden_state, seq_lens):
        tfk = tf.keras
        tfkl = tf.keras.layers
        input_shape=2
        tfpl = tfp.layers
        tfd = tfp.distributions

        tfd = tfp.distributions
        encoded_size = 16
        prior = tfd.Independent(tfd.Normal(loc=tf.zeros(encoded_size), scale=1),
                            reinterpreted_batch_ndims=1)
        tfpl = tfp.layers
        encoder = tfk.Sequential([
            tfkl.InputLayer(input_shape=input_shape),
            #tfkl.Dense(8)
            tfkl.Dense(tfpl.MultivariateNormalTriL.params_size(encoded_size),
                          activation=None),

            tfpl.MultivariateNormalTriL(
                    encoded_size,
                    activity_regularizer=tfpl.KLDivergenceRegularizer(prior, weight=1.2)),
        ])
        decoder = tfk.Sequential([
            tfkl.InputLayer(input_shape=[encoded_size]),
            tfkl.Dense(tfpl.IndependentNormal.params_size(encoded_size))
            #tfpl.IndependentBernoulli(input_shape, tfd.Bernoulli.logits)

        ])
        negative_log_likelihood = lambda x, rv_x: -rv_x.log_prob(x)
        
        x = nn.functional.relu(self.fc1(input_dict["obs_flat"].float()))
        h_in = hidden_state[0].reshape(-1, self.rnn_hidden_dim)
        h = self.rnn(x, h_in)
        q = self.fc2(h)
        return q, [h]


def _get_size(obs_space):
    return get_preprocessor(obs_space)(obs_space).size

In [67]:
class CModel(torch.nn.Module,ivy.Module):
    def __init__(self):#,obs_space,action_space,num_outputs,model_config):
        super().__init__()
        """
        TorchModelV2.__init__(
            self, obs_space, action_space, num_outputs, model_config, name
        )
        """
        self._avg_pool = torch.nn.AvgPool2d(4)
        self._linear = torch.nn.Linear(3136, 1024)
        self._relu = torch.nn.ReLU()
        #self.rnn = nn.GRUCell(self.rnn_hidden_dim, self.rnn_hidden_dim)
        #self.obs_size = _get_size(obs_space)
        #self.n_agents = model_config["n_agents"]
        
        tfk = tf.keras
        tfkl = tf.keras.layers
        input_shape=2
        tfpl = tfp.layers
        tfd = tfp.distributions

        tfd = tfp.distributions
        encoded_size = 16
        prior = tfd.Independent(tfd.Normal(loc=tf.zeros(encoded_size), scale=1),
                            reinterpreted_batch_ndims=1)
        tfpl = tfp.layers
        encoder = tfk.Sequential([
            tfkl.InputLayer(input_shape=input_shape),
            #tfkl.Dense(8)
            tfkl.Dense(tfpl.MultivariateNormalTriL.params_size(encoded_size),
                          activation=None),

            tfpl.MultivariateNormalTriL(
                    encoded_size,
                    activity_regularizer=tfpl.KLDivergenceRegularizer(prior, weight=1.2)),
        ])
        decoder = tfk.Sequential([
            tfkl.InputLayer(input_shape=[encoded_size]),
            tfkl.Dense(tfpl.IndependentNormal.params_size(encoded_size))
            #tfpl.IndependentBernoulli(input_shape, tfd.Bernoulli.logits)

        ])
        negative_log_likelihood = lambda x, rv_x: -rv_x.log_prob(x)
    #@override(ModelV2)
    def get_initial_state(self):
        # Place hidden states on same device as model.
        return [
            self.fc1.weight.new(self.n_agents, self.rnn_hidden_dim).zero_().squeeze(0)
            ]
    #@override(ModelV2)
    def _forward(self, x):

        x = self._avg_pool(x).mean(dim=-3).view(-1, 3136)
        x = self._relu(self._linear(x))
        return self._linears(x)
def _get_size(obs_space):
    return get_preprocessor(obs_space)(obs_space).size

In [8]:
#import tensorflow.compat.v2 as tf
#tf.enable_v2_behavior()
num_inducing_points=1
#this is the agent network that will be used 
#from nebulgym.decorators.torch_decorators import accelerate_model, accelerate_dataset
input_shape = [0,0]
from numba import jit
#@accelerate_model()

class RNNModel(TorchModelV2,nn.Module,ivy.Module):
  
  def __init__(self):#,obs_space, action_space, num_outputs, model_config, name): #add this in later
    
    #self.obs_size = _get_size(obs_space)
    #self.rnn_hidden_dim = model_config["lstm_cell_size"]

    """
    what do we want our VAE to do?

    our vae should: 
    1. infer a hidden state and 
    2. output an action   `

    """
    tfk = tf.keras
    tfkl = tf.keras.layers
    
    tfpl = tfp.layers
    tfd = tfp.distributions

    tfd = tfp.distributions
    encoded_size = 16
    """
    prior = tfd.Independent(tfd.Normal(loc=tf.zeros(encoded_size), scale=1),#This scale variable cannot be fixed but instead must be allowed to vary 
                                reinterpreted_batch_ndims=1)
    
    The prior should be some kind of composite or dict type distribution with multiple dirchlet distributions. 


    
    #note: the tfd Independent is for assumptions which are independent and unaffected or related to other assumptions. These distribtutions may need to be hand tuned based on domain knowledge and data analysis
    #second note: this will be the top level prior. we will have to figure out a way to ensure that this prior is only inherited from posteriors of above signals

    #solution: create an algorithm that tunes parameters of distribution and have it default to hand-written prior parameters if upper level observation space is empty and epoch is 0. 
    
    the last layer needs to be a discrete (bernoulli) distribution layer for the high and midlevel layers and a continuous distribution for the lowest layer 

    dist = tfd.Normal(loc=0.5, scale=0.25). Note: we may want to consider using the normal for all agents and just discretize for higher level agents. 

    this is going to be where we consider things like lateral disinhibition where the prior pertaining to the probability carrying out an action is going to decrease if it has already carried out an action under certain circumstances
    
    Note: we are going to replace almost all the code in the init with the compiled model laplace CModel we have above
    """
    #encoded_shape = 2
    num_schools=2
    """
    prior  = tfd.JointDistributionSequential([
        tfd.Normal(loc=0., scale=10., name="avg_effect"),  # `mu` above
        tfd.Dirichlet(2),
        tfd.Normal(loc=5., scale=1., name="avg_stddev"),  # `log(tau)` above
        tfd.Independent(tfd.Normal(loc=tf.zeros(num_schools),
                                  scale=tf.ones(num_schools),
                                  name="school_effects_standard"),  # `theta_prime` 
                        reinterpreted_batch_ndims=1)
    ])
    """
    #prior= tfd.Normal(loc=0., scale=10., name="avg_effect")
    prior = tfd.Independent(tfd.Normal(loc=tf.zeros(encoded_size), scale=1),
                        reinterpreted_batch_ndims=1)
    tfpl = tfp.layers
    encoder = tfk.Sequential([
        tfkl.InputLayer(input_shape=input_shape),
        #tfkl.Dense(8)
        tfkl.Dense(tfpl.MultivariateNormalTriL.params_size(encoded_size),
                      activation=None),

        tfpl.MultivariateNormalTriL(
                encoded_size,
                activity_regularizer=tfpl.KLDivergenceRegularizer(prior, weight=1.2)),
    ])
    decoder = tfk.Sequential([
        tfkl.InputLayer(input_shape=[encoded_size]),
        tfkl.Dense(tfpl.IndependentNormal.params_size(encoded_size))
        #tfpl.IndependentBernoulli(input_shape, tfd.Bernoulli.logits)

    ])
    negative_log_likelihood = lambda x, rv_x: -rv_x.log_prob(x)
    """
    #working theory as of 11/26/2022
    
    in order to convert from posterior -> action potential -> prior we need information theory specifically 

    probability of excitatory action potential = -log2(p(w)) if P(w) = 0.5 probability of excitatory action potential is 1

    furthermore a prediction of w being 0.5 also yields a prediction of not(w) also being 0.5 

    this is where I suspect inhibitory potentials come in since not(w) is expressed as log2(p(w)) which of 0.5 is -1 

    we will probably do this translation from the observation space and action space

    each action will actually correspond to a series of a 100 steps

    if the algorithm tries to send a signal of 0.2 bits to a particular neuronal agent we will send 
    20 excitatory action potentials and 80 0s over 100 steps  or to put it another way we will say that 0.2 

    is just the probability of it sending a excitatory potential at any given time 

    we will need to do an if else thing

    if neurons is top neuron: 
      prior = <hand coded prior>
    else: 
      if obs == 1:
        prob = obs**2 #where obs will be the observed frequency over n previously observed values
      if obs == -1:
        prob = obs**2

    """

    """
    uncomment out later

    loss = lambda y, rv_y: rv_y.variational_loss(
        y, kl_weight=np.array(batch_size, tf.float64) / x.shape[0])
    #tf.keras.optimizers.Adam(1e-4) tf.optimizers.Adam(learning_rate=0.011)
    model.compile(optimizer=tf.keras.optimizers.Adam(0.011), loss=loss)#tf.optimizers.Adam(learning_rate=0.01)
    """

    """
    we are choosing to use a variational autoencoder because there are two things that need to be done: make predictions and select action that minimizes free energy 

    
    """
    """
    decoder = tfk.Sequential([
        tfkl.InputLayer(input_shape=[encoded_size]),
        #tfpl.IndependentBernoulli(input_shape, tfd.Bernoulli.logits),
        ])
    vae = tfk.Model(inputs=encoder.inputs,
                        outputs=decoder(encoder.outputs[0]))
    """
    ivy.Module.__init__(self)
    
  #@override(ModelV2)
    @jit(nopython=True)
    def get_initial_state(self):
        # Place hidden states on same device as model.
        return [
            self.fc1.weight.new(self.n_agents, self.rnn_hidden_dim).zero_().squeeze(0)
            ]
    
    #
    #@jit(nopython=True)
    #@override(ModelV2)
    def _forward(self, input_dict, hidden_state, seq_lens):# _forward
        x = nn.functional.relu(self.fc1(input_dict["obs_flat"].float()))
        h_in = hidden_state[0].reshape(-1, self.rnn_hidden_dim)
        h = self.rnn(x, h_in)
        q = self.fc2(h)#this may be the q-value 
        return q, [h]

  
def _get_size(obs_space):
    return get_preprocessor(obs_space)(obs_space).size

ivy.set_framework('torch') 
model = RNNModel()#note that this particular line compiles if we get rid of parameters. at initialization we will call this with parameters 
#model = Laplace(model, 'regression', subset_of_weights='all', hessian_structure='full')

"""
optimized_model = optimize_model(
    model, input_data=input_data, optimization_time="constrained"
)
note: under current code minimization of kullbacker-leibler divergence is already part of the code in the form the KLD divergence regularizer which means the loss simply needs to be the "suprise" 

w
"""

C:\Users\subar\anaconda3\envs\d\lib\site-packages\ivy\framework_handler.py:115: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.10 it will stop working
  if isinstance(specific_v, collections.Hashable):


TypeError: Can't instantiate abstract class RNNModel with abstract method _forward

In [11]:
#ivy.set_framework('torch') 
model = bmodel()#CModel()#CModel()
cmodel = Laplace(model,'regression', subset_of_weights='last_layer', hessian_structure='full')#all

#this compiles under if modelV2 is not passed wiht any arguments. It demands arguments when modelV2 is passed

TypeError: __init__() missing 5 required positional arguments: 'obs_space', 'action_space', 'num_outputs', 'model_config', and 'name'

In [9]:
print(cmodel)

In [ ]:
#put this code into 

model = optimize_model(
  cmodel, input_data=input_data, device="gpu"
)

As of january 13 2022 we have performed a sucessful partial execution test of the project pegasus agent 
network

We still have to determine the following:
1. how to create heterogenuous agents
2. how to visualize using pysurfer
3. figure out what if any alterations will be needed for the mixer network 

ANS for 3: we should perhaps think of the mixer network as maximizing the overall free energy and 
acting on the level of active inference instead of predictive coding. 

we are going to use laplace-torch for among other things model selection

This means that after we finish pretraining our RL algorithm on normal conditions we will fine tune and select simulated models generated by our RL algorithm by how well they fit the frieburg data. 

we are actually only going to have 1 env. I dont think we need the brain env only the humanoid env




In [35]:
# Torch must be installed.
#torch, nn = try_import_torch(error=True)

#logger = logging.getLogger(__name__)


class QMixLoss(nn.Module):
    def __init__(
        self,
        model,
        target_model,
        mixer,
        target_mixer,
        n_agents,
        n_actions,
        double_q=True,
        gamma=0.99,
    ):
        nn.Module.__init__(self)
        self.model = model
        self.target_model = target_model
        self.mixer = mixer
        self.target_mixer = target_mixer
        self.n_agents = n_agents
        self.n_actions = n_actions
        self.double_q = double_q
        self.gamma = gamma

    def forward(
        self,
        rewards,
        actions,
        terminated,
        mask,
        obs,
        next_obs,
        action_mask,
        next_action_mask,
        state=None,
        next_state=None,
    ):
        """Forward pass of the loss.
        Args:
            rewards: Tensor of shape [B, T, n_agents]
            actions: Tensor of shape [B, T, n_agents]
            terminated: Tensor of shape [B, T, n_agents]
            mask: Tensor of shape [B, T, n_agents]
            obs: Tensor of shape [B, T, n_agents, obs_size]
            next_obs: Tensor of shape [B, T, n_agents, obs_size]
            action_mask: Tensor of shape [B, T, n_agents, n_actions]
            next_action_mask: Tensor of shape [B, T, n_agents, n_actions]
            state: Tensor of shape [B, T, state_dim] (optional)
            next_state: Tensor of shape [B, T, state_dim] (optional)
        """

        # Assert either none or both of state and next_state are given
        if state is None and next_state is None:
            state = obs  # default to state being all agents' observations
            next_state = next_obs
        elif (state is None) != (next_state is None):
            raise ValueError(
                "Expected either neither or both of `state` and "
                "`next_state` to be given. Got: "
                "\n`state` = {}\n`next_state` = {}".format(state, next_state)
            )

        # Calculate estimated Q-Values
        mac_out = _unroll_mac(self.model, obs)

        # Pick the Q-Values for the actions taken -> [B * n_agents, T]
        chosen_action_qvals = torch.gather(
            mac_out, dim=3, index=actions.unsqueeze(3)
        ).squeeze(3)

        # Calculate the Q-Values necessary for the target
        target_mac_out = _unroll_mac(self.target_model, next_obs)

        # Mask out unavailable actions for the t+1 step
        ignore_action_tp1 = (next_action_mask == 0) & (mask == 1).unsqueeze(-1)
        target_mac_out[ignore_action_tp1] = -np.inf

        # Max over target Q-Values
        if self.double_q:
            # Double Q learning computes the target Q values by selecting the
            # t+1 timestep action according to the "policy" neural network and
            # then estimating the Q-value of that action with the "target"
            # neural network
            """
            note that the target neural network uses the RNN network defined above and that 

            policy neural network also appears to be the mixer network 

            double q-learning will find 2 q values. One will be used in our case to identify an action that minimizes free energy while another 
            will be used to evaluate that action 
            """

            # Compute the t+1 Q-values to be used in action selection
            # using next_obs
            mac_out_tp1 = _unroll_mac(self.model, next_obs)

            # mask out unallowed actions
            mac_out_tp1[ignore_action_tp1] = -np.inf

            # obtain best actions at t+1 according to policy NN
            cur_max_actions = mac_out_tp1.argmax(dim=3, keepdim=True)

            # use the target network to estimate the Q-values of policy
            # network's selected actions
            target_max_qvals = torch.gather(target_mac_out, 3, cur_max_actions).squeeze(
                3
            )
        else:
            target_max_qvals = target_mac_out.max(dim=3)[0]

        assert (
            target_max_qvals.min().item() != -np.inf
        ), "target_max_qvals contains a masked action; \
            there may be a state with no valid actions."

        # Mix
        if self.mixer is not None:
            chosen_action_qvals = self.mixer(chosen_action_qvals, state)
            target_max_qvals = self.target_mixer(target_max_qvals, next_state)

        # Calculate 1-step Q-Learning targets
        targets = rewards + self.gamma * (1 - terminated) * target_max_qvals

        # Td-error
        td_error = chosen_action_qvals - targets.detach()

        mask = mask.expand_as(td_error)

        # 0-out the targets that came from padded data
        masked_td_error = td_error * mask

        # Normal L2 loss, take mean over actual data
        loss = (masked_td_error ** 2).sum() / mask.sum()
        return loss, mask, masked_td_error, chosen_action_qvals, targets


class QMixTorchPolicy(TorchPolicy):
    """QMix impl. Assumes homogeneous agents for now.
    You must use MultiAgentEnv.with_agent_groups() to group agents
    together for QMix. This creates the proper Tuple obs/action spaces and
    populates the '_group_rewards' info field.
    Action masking: to specify an action mask for individual agents, use a
    dict space with an action_mask key, e.g. {"obs": ob, "action_mask": mask}.
    The mask space must be `Box(0, 1, (n_actions,))`.
    """

    def __init__(self, obs_space, action_space, config):
        _validate(obs_space, action_space)
        config = dict(ray.rllib.algorithms.qmix.qmix.DEFAULT_CONFIG, **config)
        self.framework = "torch"

        self.n_agents = len(obs_space.original_space.spaces)
        config["model"]["n_agents"] = self.n_agents
        self.n_actions = action_space.spaces[0].n
        self.h_size = config["model"]["lstm_cell_size"]
        self.has_env_global_state = False
        self.has_action_mask = False

        agent_obs_space = obs_space.original_space.spaces[0]
        if isinstance(agent_obs_space, gym.spaces.Dict):
            space_keys = set(agent_obs_space.spaces.keys())
            if "obs" not in space_keys:
                raise ValueError("Dict obs space must have subspace labeled `obs`")
            self.obs_size = _get_size(agent_obs_space.spaces["obs"])
            if "action_mask" in space_keys:
                mask_shape = tuple(agent_obs_space.spaces["action_mask"].shape)
                if mask_shape != (self.n_actions,):
                    raise ValueError(
                        "Action mask shape must be {}, got {}".format(
                            (self.n_actions,), mask_shape
                        )
                    )
                self.has_action_mask = True
            if ENV_STATE in space_keys:
                self.env_global_state_shape = _get_size(
                    agent_obs_space.spaces[ENV_STATE]
                )
                self.has_env_global_state = True
            else:
                self.env_global_state_shape = (self.obs_size, self.n_agents)
            # The real agent obs space is nested inside the dict
            config["model"]["full_obs_space"] = agent_obs_space
            agent_obs_space = agent_obs_space.spaces["obs"]
        else:
            self.obs_size = _get_size(agent_obs_space)
            self.env_global_state_shape = (self.obs_size, self.n_agents)
            
            
        ivy.set_framework('torch') 
        model = bmodel()#CModel()#CModel()
        cmodel = Laplace(model,'regression', subset_of_weights='last_layer', hessian_structure='full')#all
        
        self.model = ModelCatalog.get_model_v2(
            agent_obs_space,
            action_space.spaces[0],
            self.n_actions,
            config["model"],
            framework="torch",
            name="model",
            default_model=cmodel#RNNModel,
        )

        super().__init__(obs_space, action_space, config, model=self.model)

        self.target_model = ModelCatalog.get_model_v2(
            agent_obs_space,
            action_space.spaces[0],
            self.n_actions,
            config["model"],
            framework="torch",
            name="target_model",
            default_model=cmodel#CModel#RNNModel,#this target model is the agent model 
        ).to(self.device)

        self.exploration = self._create_exploration()

        # Setup the mixer network.
        if config["mixer"] is None:
            self.mixer = None
            self.target_mixer = None
        elif config["mixer"] == "qmix":
            self.mixer = QMixer(
                self.n_agents, self.env_global_state_shape, config["mixing_embed_dim"]
            ).to(self.device)
            self.target_mixer = QMixer(
                self.n_agents, self.env_global_state_shape, config["mixing_embed_dim"]
            ).to(self.device)
        elif config["mixer"] == "vdn":
            self.mixer = VDNMixer().to(self.device)
            self.target_mixer = VDNMixer().to(self.device)
        else:
            raise ValueError("Unknown mixer type {}".format(config["mixer"]))

        self.cur_epsilon = 1.0
        self.update_target()  # initial sync

        # Setup optimizer
        self.params = list(self.model.parameters())
        if self.mixer:
            self.params += list(self.mixer.parameters())
        self.loss = QMixLoss(
            self.model,
            self.target_model,
            self.mixer,
            self.target_mixer,
            self.n_agents,
            self.n_actions,
            self.config["double_q"],
            self.config["gamma"],
        )
        from torch.optim import RMSprop
        #it appears that this RMSprop is optimizing the mixer network and not the agent network
        self.rmsprop_optimizer = RMSprop(
            params=self.params,
            lr=config["lr"],
            alpha=config["optim_alpha"],
            eps=config["optim_eps"],
        )

    @override(TorchPolicy)
    def compute_actions_from_input_dict(
        self,
        input_dict: Dict[str, TensorType],
        explore: bool = None,
        timestep: Optional[int] = None,
        **kwargs,
    ) -> Tuple[TensorType, List[TensorType], Dict[str, TensorType]]:

        obs_batch = input_dict[SampleBatch.OBS]
        state_batches = []
        i = 0
        while f"state_in_{i}" in input_dict:
            state_batches.append(input_dict[f"state_in_{i}"])
            i += 1

        explore = explore if explore is not None else self.config["explore"]
        obs_batch, action_mask, _ = self._unpack_observation(obs_batch)
        # We need to ensure we do not use the env global state
        # to compute actions

        # Compute actions
        with torch.no_grad():
            q_values, hiddens = _mac(
                self.model,
                torch.as_tensor(obs_batch, dtype=torch.float, device=self.device),
                [
                    torch.as_tensor(np.array(s), dtype=torch.float, device=self.device)
                    for s in state_batches
                ],
            )
            avail = torch.as_tensor(action_mask, dtype=torch.float, device=self.device)
            masked_q_values = q_values.clone()
            masked_q_values[avail == 0.0] = -float("inf")
            masked_q_values_folded = torch.reshape(
                masked_q_values, [-1] + list(masked_q_values.shape)[2:]
            )
            actions, _ = self.exploration.get_exploration_action(
                action_distribution=TorchCategorical(masked_q_values_folded),
                timestep=timestep,
                explore=explore,
            )
            actions = (
                torch.reshape(actions, list(masked_q_values.shape)[:-1]).cpu().numpy()
            )
            hiddens = [s.cpu().numpy() for s in hiddens]

        return tuple(actions.transpose([1, 0])), hiddens, {}

    @override(TorchPolicy)
    def compute_actions(self, *args, **kwargs):
        return self.compute_actions_from_input_dict(*args, **kwargs)

    @override(TorchPolicy)
    def compute_log_likelihoods(
        self,
        actions,
        obs_batch,
        state_batches=None,
        prev_action_batch=None,
        prev_reward_batch=None,
    ):
        obs_batch, action_mask, _ = self._unpack_observation(obs_batch)
        return np.zeros(obs_batch.size()[0])

    @override(TorchPolicy)
    def learn_on_batch(self, samples):
        obs_batch, action_mask, env_global_state = self._unpack_observation(
            samples[SampleBatch.CUR_OBS]
        )
        (
            next_obs_batch,
            next_action_mask,
            next_env_global_state,
        ) = self._unpack_observation(samples[SampleBatch.NEXT_OBS])
        group_rewards = self._get_group_rewards(samples[SampleBatch.INFOS])

        input_list = [
            group_rewards,
            action_mask,
            next_action_mask,
            samples[SampleBatch.ACTIONS],
            samples[SampleBatch.DONES],
            obs_batch,
            next_obs_batch,
        ]
        if self.has_env_global_state:
            input_list.extend([env_global_state, next_env_global_state])

        output_list, _, seq_lens = chop_into_sequences(
            episode_ids=samples[SampleBatch.EPS_ID],
            unroll_ids=samples[SampleBatch.UNROLL_ID],
            agent_indices=samples[SampleBatch.AGENT_INDEX],
            feature_columns=input_list,
            state_columns=[],  # RNN states not used here
            max_seq_len=self.config["model"]["max_seq_len"],
            dynamic_max=True,
        )
        # These will be padded to shape [B * T, ...]
        if self.has_env_global_state:
            (
                rew,
                action_mask,
                next_action_mask,
                act,
                dones,
                obs,
                next_obs,
                env_global_state,
                next_env_global_state,
            ) = output_list
        else:
            (
                rew,
                action_mask,
                next_action_mask,
                act,
                dones,
                obs,
                next_obs,
            ) = output_list
        B, T = len(seq_lens), max(seq_lens)

        def to_batches(arr, dtype):
            new_shape = [B, T] + list(arr.shape[1:])
            return torch.as_tensor(
                np.reshape(arr, new_shape), dtype=dtype, device=self.device
            )

        rewards = to_batches(rew, torch.float)
        actions = to_batches(act, torch.long)
        obs = to_batches(obs, torch.float).reshape([B, T, self.n_agents, self.obs_size])
        action_mask = to_batches(action_mask, torch.float)
        next_obs = to_batches(next_obs, torch.float).reshape(
            [B, T, self.n_agents, self.obs_size]
        )
        next_action_mask = to_batches(next_action_mask, torch.float)
        if self.has_env_global_state:
            env_global_state = to_batches(env_global_state, torch.float)
            next_env_global_state = to_batches(next_env_global_state, torch.float)

        # TODO(ekl) this treats group termination as individual termination
        terminated = (
            to_batches(dones, torch.float).unsqueeze(2).expand(B, T, self.n_agents)
        )

        # Create mask for where index is < unpadded sequence length
        filled = np.reshape(
            np.tile(np.arange(T, dtype=np.float32), B), [B, T]
        ) < np.expand_dims(seq_lens, 1)
        mask = (
            torch.as_tensor(filled, dtype=torch.float, device=self.device)
            .unsqueeze(2)
            .expand(B, T, self.n_agents)
        )

        # Compute loss
        loss_out, mask, masked_td_error, chosen_action_qvals, targets = self.loss(
            rewards,
            actions,
            terminated,
            mask,
            obs,
            next_obs,
            action_mask,
            next_action_mask,
            env_global_state,
            next_env_global_state,
        )

        # Optimise
        self.rmsprop_optimizer.zero_grad()
        loss_out.backward()
        grad_norm_info = apply_grad_clipping(self, self.rmsprop_optimizer, loss_out)
        self.rmsprop_optimizer.step()

        mask_elems = mask.sum().item()
        stats = {
            "loss": loss_out.item(),
            "td_error_abs": masked_td_error.abs().sum().item() / mask_elems,
            "q_taken_mean": (chosen_action_qvals * mask).sum().item() / mask_elems,
            "target_mean": (targets * mask).sum().item() / mask_elems,
        }
        stats.update(grad_norm_info)

        return {LEARNER_STATS_KEY: stats}

    @override(TorchPolicy)
    def get_initial_state(self):  # initial RNN state
        return [
            s.expand([self.n_agents, -1]).cpu().numpy()
            for s in self.model.get_initial_state()
        ]

    @override(TorchPolicy)
    def get_weights(self):
        return {
            "model": self._cpu_dict(self.model.state_dict()),
            "target_model": self._cpu_dict(self.target_model.state_dict()),
            "mixer": self._cpu_dict(self.mixer.state_dict()) if self.mixer else None,
            "target_mixer": self._cpu_dict(self.target_mixer.state_dict())
            if self.mixer
            else None,
        }

    @override(TorchPolicy)
    def set_weights(self, weights):
        self.model.load_state_dict(self._device_dict(weights["model"]))
        self.target_model.load_state_dict(self._device_dict(weights["target_model"]))
        if weights["mixer"] is not None:
            self.mixer.load_state_dict(self._device_dict(weights["mixer"]))
            self.target_mixer.load_state_dict(
                self._device_dict(weights["target_mixer"])
            )


    @override(TorchPolicy)
    def get_state(self):
        state = self.get_weights()
        state["cur_epsilon"] = self.cur_epsilon
        return state

    @override(TorchPolicy)
    def set_state(self, state):
        self.set_weights(state)
        self.set_epsilon(state["cur_epsilon"])

    def update_target(self):
        self.target_model.load_state_dict(self.model.state_dict())
        if self.mixer is not None:
            self.target_mixer.load_state_dict(self.mixer.state_dict())
        logger.debug("Updated target networks")

    def set_epsilon(self, epsilon):
        self.cur_epsilon = epsilon

    def _get_group_rewards(self, info_batch):
        group_rewards = np.array(
            [info.get(GROUP_REWARDS, [0.0] * self.n_agents) for info in info_batch]
        )
        return group_rewards

    def _device_dict(self, state_dict):
        return {
            k: torch.as_tensor(v, device=self.device) for k, v in state_dict.items()
        }

    @staticmethod
    def _cpu_dict(state_dict):
        return {k: v.cpu().detach().numpy() for k, v in state_dict.items()}

    def _unpack_observation(self, obs_batch):
        """Unpacks the observation, action mask, and state (if present)
        from agent grouping.
        Returns:
            obs (np.ndarray): obs tensor of shape [B, n_agents, obs_size]
            mask (np.ndarray): action mask, if any
            state (np.ndarray or None): state tensor of shape [B, state_size]
                or None if it is not in the batch
        """

        unpacked = _unpack_obs(
            np.array(obs_batch, dtype=np.float32),
            self.observation_space.original_space,
            tensorlib=np,
        )

        if isinstance(unpacked[0], dict):
            assert "obs" in unpacked[0]
            unpacked_obs = [np.concatenate(tree.flatten(u["obs"]), 1) for u in unpacked]
        else:
            unpacked_obs = unpacked

        obs = np.concatenate(unpacked_obs, axis=1).reshape(
            [len(obs_batch), self.n_agents, self.obs_size]
        )

        if self.has_action_mask:
            action_mask = np.concatenate(
                [o["action_mask"] for o in unpacked], axis=1
            ).reshape([len(obs_batch), self.n_agents, self.n_actions])
        else:
            action_mask = np.ones(
                [len(obs_batch), self.n_agents, self.n_actions], dtype=np.float32
            )

        if self.has_env_global_state:
            state = np.concatenate(tree.flatten(unpacked[0][ENV_STATE]), 1)
        else:
            state = None
        return obs, action_mask, state


def _validate(obs_space, action_space):
    if not hasattr(obs_space, "original_space") or not isinstance(
        obs_space.original_space, gym.spaces.Tuple
    ):
        raise ValueError(
            "Obs space must be a Tuple, got {}. Use ".format(obs_space)
            + "MultiAgentEnv.with_agent_groups() to group related "
            "agents for QMix."
        )
    if not isinstance(action_space, gym.spaces.Tuple):
        raise ValueError(
            "Action space must be a Tuple, got {}. ".format(action_space)
            + "Use MultiAgentEnv.with_agent_groups() to group related "
            "agents for QMix."
        )
    if not isinstance(action_space.spaces[0], gym.spaces.Discrete):
        raise ValueError(
            "QMix requires a discrete action space, got {}".format(
                action_space.spaces[0]
            )
        )
    if len({str(x) for x in obs_space.original_space.spaces}) > 1:
        raise ValueError(
            "Implementation limitation: observations of grouped agents "
            "must be homogeneous, got {}".format(obs_space.original_space.spaces)
        )
    if len({str(x) for x in action_space.spaces}) > 1:
        raise ValueError(
            "Implementation limitation: action space of grouped agents "
            "must be homogeneous, got {}".format(action_space.spaces)
        )


def _mac(model, obs, h):
    """Forward pass of the multi-agent controller.
    Args:
        model: TorchModelV2 class
        obs: Tensor of shape [B, n_agents, obs_size]
        h: List of tensors of shape [B, n_agents, h_size]
    Returns:
        q_vals: Tensor of shape [B, n_agents, n_actions]
        h: Tensor of shape [B, n_agents, h_size]
    """
    B, n_agents = obs.size(0), obs.size(1)
    if not isinstance(obs, dict):
        obs = {"obs": obs}
    obs_agents_as_batches = {k: _drop_agent_dim(v) for k, v in obs.items()}
    h_flat = [s.reshape([B * n_agents, -1]) for s in h]
    
    model = optimize_model(
      model, input_data=obs, device="gpu"
    )
    
    
    q_flat, h_flat = model(obs_agents_as_batches, h_flat, None)
    return q_flat.reshape([B, n_agents, -1]), [
        s.reshape([B, n_agents, -1]) for s in h_flat
    ]




def _unroll_mac(model, obs_tensor):
    """Computes the estimated Q values for an entire trajectory batch"""
    B = obs_tensor.size(0)
    T = obs_tensor.size(1)
    n_agents = obs_tensor.size(2)
    """
    model = optimize_model(
      cmodel, input_data=input_data, device="gpu"
    )
    
    """

    mac_out = []
    h = [s.expand([B, n_agents, -1]) for s in model.get_initial_state()]
    for t in range(T):
        q, h = _mac(model, obs_tensor[:, t], h)
        mac_out.append(q)
    mac_out = torch.stack(mac_out, dim=1)  # Concat over time

    return mac_out


def _drop_agent_dim(T):
    shape = list(T.shape)
    B, n_agents = shape[0], shape[1]
    return T.reshape([B * n_agents] + shape[2:])


def _add_agent_dim(T, n_agents):
    shape = list(T.shape)
    B = shape[0] // n_agents
    assert shape[0] % n_agents == 0
    return T.reshape([B, n_agents] + shape[1:])

TypeError: <class 'gymnasium.spaces.dict.Dict'> is not a generic class

Everything up to this point has sucessfully executed

if prediction errors are what it is sending up and predictions are what it is sending down what is it sending to neurons
to its side?

ANS: most likely predictions but we may need to check this

In the active inference book it shows that the highest level neurons and their expectations conditioned on policy has no 
lateral connections while the lower level neurons which are represented with average under policy do have lateral connections

The lowest level neurons are represented just as average under policies. 

New hypothesis: it may just be excitatory or inhibitory signals that are being sent laterally. (see phenomena such as lateral inhibition. 

In addition we should consider bringing back at this stage temporal and spatial summation. We know now that the upper level neuronal agents form discrete models and only the lowest level neurons send and recieve continuous signals. 

Question: Could lateral inhibition be related to that sampling method in QUASI? 

Note: empirical priors from high level neuronal agents drive habits (which can be seen as analogous to model-free RL) and influence policy selection independently of expected free energy. while The expected free energy of a policy that comes from predicted outcomes and prediction error drives goal directed behavior and is analogous to model-based RL. Low dopamine favors context sensitive priors in the indirect pathways whose role is to suppress implausible policies. (it favors habits over goal directed behavior). Complete depletion fo dopamine leads to an inability to enact specific policies.

we can hold beliefs not just about states of the world but also about the fixed or slowly varying parameters that determine dependencies between variables. The substrate of these beliefs is the efficacy of synaptic connections representing time-varying variables. When we observe an outcome that we believe was generated by a given state we can update beliefs about the parameter connecting the two, reflecting an increase in the probability of them occuring in the future 

Translation: we will also need to infer not just states of the world but also parameters that determine dependenceis between variables (which in this case are different neurons or populations of neurons)

where teh lowest level might deal with the requisite changes in muscle length descending input is based on decisions about which movement to make

connections entering and leaving a cotrical column relate to likelhood distributions wheras transition probabilities and continuous dynamics depend on connections within a microcircuit

This suggests that learning dynamics should lead to changes in intrninsic connectivity while learning observations should modify extrinsic connectivity. 

question for the future: Does lateral inhibition play an analogous role as the XOR parity constraints do in the WISH scheme? If does a WISH like mechanism play a role in creating the low quality approximate attention schema that creates consciousness?

parameters to of interest: prior precision (already built in to and taken care of by laplace torch, connection parameter will need to figure out how to handle this. 

what if we put the other parameters like the connection parameter into the local reward function? 

our agent network needs to:
1. infer the hidden state 
2. generate a prediction to send down the hierarchy or an action to send laterally to another neuronal agent
3. also needs to infer the parameterization that determines dependencies between neuronal agents

I dont believe that our variational autoencoder in its present state can do all three things. we need to find some way to figure these things out sequentially by first inferring hiddens tates then finding the parameterization and finally generating the predictions and actions. Our variational autoencoder is only configured to 1 and 3 and partially 2 if combined with laplace-torch

Apparently a hidden markov model is a special case of POMDP in which choices and and behavior are ignored. The first half of our agent autoencoder may need to act like a hidden markov model. 

For first term of the global reward function see the optimal experimental design in pyro. Apparently the first term of minimzing free energy has the same exact form as this. The second term is expected ambiguity where 

qmix uses double q learning where it learns 2 q values: one for choosing the action and a second for evaluating the action



In [13]:

from ray.rllib.env.wrappers.group_agents_wrapper import GroupAgentsWrapper
class brain(MultiAgentEnv):    
    def __init__(self, config):
        self.num_agents = 3
        """
        this is how the actions and actions space will work: neurons do either spatial or temporal summation depending on
        whether they are getting repeated inputs from on other neuron or inputs from multiple neurons.

        The rate of fire of the neuron for specific impulses to the external agent will determine how much to slow
        things down in the acceleration game or how much to speed things up. 

        There will be a baseline probability of firing is 0.5 
        and in order to collectively minimize Free energy the neuronal agents will have to increase or decrease 
        the probability of firing 

        so the action space cannot be encoded as just one discrete thing. It will have to be a dict composed of:
        box(probability between -1 ans 1 of firing) and discrete(num_neuronal_agents)

        """
        self._spaces_in_preferred_format = True
        self.a = self.num_agents-1#Box(low=-1.0, high=1.0, shape=(2,), dtype=np.float32)
        #the two kind of actions any agent can do are action potentials and post-synaptic potentials
        #in addition the agent neuron can direct their action to any one of the neurons that exist
        self.action_space = gym.spaces.Dict(
            {
                "agent0": Dict(
                    {
                        #"prob": Discrete(100), 
                        "action_potential": Discrete(1) #external agent neurons
                    },dtype=np.float32),
                "agent1": Dict(
                    {
                        #"prob": Discrete(100),
                        "action_potential": Discrete(1) #excitatory neurons
                    } ,dtype=np.float32),
                "agent2": Dict(
                    {
                        #"prob": Discrete(100),
                        "outsideAction":gym.spaces.Box(low=-1.0, high=1.0, shape=(10,)),
                        "action_potential": Discrete(1) #inhibitory neurons
                    }, dtype=np.float32)
            }
        )
        self.observation_space = gym.spaces.Dict(
                {
                    "agent0": self.action_space,#agent 0 will be our external agent/motor neurons
                    "agent1": self.action_space,#Discrete(1), #excitatory neurons
                    "agent2": gym.spaces.Box(low=0, high=1.0, shape=(10,))#Discrete(1)#inhibitory neurons
                }
            )


     #config.get(Dict({"prob": Discrete(200),"pos": Discrete(self.a),"action_potential": Discrete(2),"ps_potential":Box(low=-1.0, high=1.0, shape=(2,), dtype=np.float32))}))

        #external agent action space needs to involve observation = accesible_state*d where d is the distortion variable 

        #the observation space is only what the neuronal agent recievs from the other neuronal agent
        #self.observation_space = config.get(Discrete(self.a))
        self.timestep_limit = config.get("ts", 1000)

        """ Create the A matrix  """
        """
        A = np.zeros( (n_states, n_observations))
        np.fill_diagonal(A, 0.5) 
        log_likelihood = log_stable(A[observation_index,:])

        log_prior = log_stable(prior)

        qs = softmax(log_likelihood + log_prior)

        qs_past = utils.onehot(4, n_states) # agent believes they were at location 4 -- i.e. (1,1) one timestep ag
        """

        """

        A-matrix here is about the conditional distribution between observations and the hidden state. The hidden state being
        the action corresponding to a change in the acceleration number needed for the acceleration number to become 0 

        The b-matrix is probability of current state given past state and past action. 

        if state is synapse strength then what we need to do is say, synapse strength depends on past synapse strength 
        and whether or not the neuron fired a positive signal at t-1, an inhibitory signal or did not fire at all.

        THe second factor which will go into state is the optimal acceleration number. 

        c-matrix for the external hidden state factor will be a uniform prior across all possible acceleration numbers. Furthermore the 
        d-matrix for the external state factor will be a uniform prior across all possible acceleration numbers. 


        """

    def reset(self):
        """Resets the episode and returns the initial observation of the new one.
        # Reset the episode len.
        self.episode_len = 0
        # Sample a random number from our observation space. in the line directly below is where we should do A 
        self.cur_obs = self.observation_space.sample()#qo_u_left = get_expected_observations(A, qs_u_left)
        # Return initial observation.
        return self.cur_obs
        """
        self.dones = set()
        return {i: self.observation_space[i].sample() for i in self.agents}

    def step(self, action):        
        obs, rew, done, info = {}, {}, {}, {}

        if self.actions_are_logits:
            action_dict = {
                k: np.random.choice([0, 1], p=v) for k, v in action_dict.items()
            }

            state_index = np.flatnonzero(self.state)
        if state_index == 0:
            action = action_dict[self.agent_1]
            assert action in [0, 1], action
            if action == 0:
                self.state = np.array([0, 1, 0])
            else:
                self.state = np.array([0, 0, 1])
            global_rew = 0
            terminated = False
        elif state_index == 1:
            global_rew = 7
            terminated = True
        else:
            if action_dict[self.agent_1] == 0 and action_dict[self.agent_2] == 0:
                global_rew = 0
            elif action_dict[self.agent_1] == 1 and action_dict[self.agent_2] == 1:
                global_rew = 8
            else:
                global_rew = 1
            terminated = True
        global_rew = 1 #put in overall active inference reward here 
        rewards = {self.agent_0: global_rew /2, self.agent_1: global_rew / 2.0, self.agent_2: global_rew / 2.0} #modify global rew for each group of agents

        for i, action in action_dict.items():
            obs[i] = self.observation_space[i].sample()
            rew[i] = 0.0
            done[i] = False
            info[i] = {}
            done["__all__"] = len(self.dones) == len(self.agents)
        return obs, rew, done, info
        #this ensures that the agents are not all homogenous 
    def with_agent_groups(
        self,
        groups: Dict[str, List[AgentID]],
        obs_space: gym.Space = None,
            act_space: gym.Space = None) -> "MultiAgentEnv":
        """Convenience method for grouping together agents in this env.

        An agent group is a list of agent IDs that are mapped to a single
        logical agent. All agents of the group must act at the same time in the
        environment. The grouped agent exposes Tuple action and observation
        spaces that are the concatenated action and obs spaces of the
        individual agents.

        The rewards of all the agents in a group are summed. The individual
        agent rewards are available under the "individual_rewards" key of the
        group info return.

        Agent grouping is required to leverage algorithms such as Q-Mix.

        Args:
            groups: Mapping from group id to a list of the agent ids
                of group members. If an agent id is not present in any group
                value, it will be left ungrouped. The group id becomes a new agent ID
                in the final environment.
            obs_space: Optional observation space for the grouped
                env. Must be a tuple space. If not provided, will infer this to be a
                Tuple of n individual agents spaces (n=num agents in a group).
            act_space: Optional action space for the grouped env.
                Must be a tuple space. If not provided, will infer this to be a Tuple
                of n individual agents spaces (n=num agents in a group).

        Examples:
            >>> from ray.rllib.env.multi_agent_env import MultiAgentEnv
            >>> class MyMultiAgentEnv(MultiAgentEnv): # doctest: +SKIP
            ...     # define your env here
            ...     ... # doctest: +SKIP
            >>> env = MyMultiAgentEnv(...) # doctest: +SKIP
            >>> grouped_env = env.with_agent_groups(env, { # doctest: +SKIP
            ...   "group1": ["agent1", "agent2", "agent3"], # doctest: +SKIP
            ...   "group2": ["agent4", "agent5"], # doctest: +SKIP
            ... }) # doctest: +SKIP
        """


        return GroupAgentsWrapper(self, groups, obs_space, act_space)

NameError: name 'AgentID' is not defined

We are going to have two parallel vectorized parallel enviroments. 

In [ ]:
    """    
    config = (
            PPOConfig()
            .environment(HierarchicalWindyMazeEnv)
            .framework(args.framework)
            .rollouts(num_rollout_workers=0)
            .training(entropy_coeff=0.01)
            .multi_agent(
                policies={
                    "high_level_policy": (
                        None,
                        maze.observation_space,
                        Discrete(4),
                        PPOConfig.overrides(gamma=0.9),
                    ),
                    "low_level_policy": (
                        None,
                        Tuple([maze.observation_space, Discrete(4)]),
                        maze.action_space,
                        PPOConfig.overrides(gamma=0.0),
                    ),
                },
                policy_mapping_fn=policy_mapping_fn,
            )
            # Use GPUs iff `RLLIB_NUM_GPUS` env var set to > 0.
            .resources(num_gpus=int(os.environ.get("RLLIB_NUM_GPUS", "0")))
            
        )
        """

In [14]:
class QMixConfig(SimpleQConfig):
    """Defines a configuration class from which QMix can be built.
    Example:
        >>> from ray.rllib.examples.env.two_step_game import TwoStepGame
        >>> from ray.rllib.algorithms.qmix import QMixConfig
        >>> config = QMixConfig().training(gamma=0.9, lr=0.01, kl_coeff=0.3)\
        ...             .resources(num_gpus=0)\
        ...             .rollouts(num_workers=4)
        >>> print(config.to_dict())
        >>> # Build an Algorithm object from the config and run 1 training iteration.
        >>> algo = config.build(env=TwoStepGame)
        >>> algo.train()
    Example:
        >>> from ray.rllib.examples.env.two_step_game import TwoStepGame
        >>> from ray.rllib.algorithms.qmix import QMixConfig
        >>> from ray import tune
        >>> config = QMixConfig()
        >>> # Print out some default values.
        >>> print(config.optim_alpha)
        >>> # Update the config object.
        >>> config.training(lr=tune.grid_search([0.001, 0.0001]), optim_alpha=0.97)
        >>> # Set the config object's env.
        >>> config.environment(env=TwoStepGame)
        >>> # Use to_dict() to get the old-style python config dict
        >>> # when running with tune.
        >>> tune.run(
        ...     "QMix",
        ...     stop={"episode_reward_mean": 200},
        ...     config=config.to_dict(),
        ... )
    """

    def __init__(self):
        """Initializes a PPOConfig instance."""
        super().__init__(algo_class=QMix)

        # fmt: off
        # __sphinx_doc_begin__
        # QMix specific settings:
        self.mixer = "qmix"
        self.mixing_embed_dim = 32
        self.double_q = True
        self.optim_alpha = 0.99
        self.optim_eps = 0.00001
        self.grad_clip = 10

        # Override some of AlgorithmConfig's default values with QMix-specific values.
        # .training()
        self.lr = 0.0005
        self.train_batch_size = 32
        self.target_network_update_freq = 500
        # Number of timesteps to collect from rollout workers before we start
        # sampling from replay buffers for learning. Whether we count this in agent
        # steps  or environment steps depends on config["multiagent"]["count_steps_by"].
        self.num_steps_sampled_before_learning_starts = 1000
        self.replay_buffer_config = {
            "type": "ReplayBuffer",
            # Specify prioritized replay by supplying a buffer type that supports
            # prioritization, for example: MultiAgentPrioritizedReplayBuffer.
            "prioritized_replay": DEPRECATED_VALUE,
            # Size of the replay buffer in batches (not timesteps!).
            "capacity": 1000,
            
            # Choosing `fragments` here makes it so that the buffer stores entire
            # batches, instead of sequences, episodes or timesteps.
            "storage_unit": "fragments",
            # Whether to compute priorities on workers.
            "worker_side_prioritization": False,
        }
        self.model = {
            "lstm_cell_size": 64,
            "max_seq_len": 999999,
        }

        # .framework()
        self.framework_str = "torch"

        # .rollouts()
        self.num_workers = 3
        self.rollout_fragment_length = 4
        self.batch_mode = "complete_episodes"

        # .reporting()
        self.min_time_s_per_iteration = 1
        self.min_sample_timesteps_per_iteration = 1000

        # .exploration()
        self.exploration_config = {
            # The Exploration class to use.
            "type": "EpsilonGreedy",
            # Config for the Exploration class' constructor:
            "initial_epsilon": 1.0,
            "final_epsilon": 0.01,
            # Timesteps over which to anneal epsilon.
            "epsilon_timesteps": 40000,

            # For soft_q, use:
            # "exploration_config" = {
            #   "type": "SoftQ"
            #   "temperature": [float, e.g. 1.0]
            # }
        }

        # .evaluation()
        # Evaluate with epsilon=0 every `evaluation_interval` training iterations.
        # The evaluation stats will be reported under the "evaluation" metric key.
        # Note that evaluation is currently not parallelized, and that for Ape-X
        # metrics are already only reported for the lowest epsilon workers.
        self.evaluation_interval = None
        self.evaluation_duration = 10
        self.evaluation_config = {
            "explore": False,
        }
        # __sphinx_doc_end__
        # fmt: on

        self.worker_side_prioritization = DEPRECATED_VALUE

    @override(SimpleQConfig)
    def training(
        self,
        *,
        mixer: Optional[str] = None,
        mixing_embed_dim: Optional[int] = None,
        double_q: Optional[bool] = None,
        target_network_update_freq: Optional[int] = None,
        replay_buffer_config: Optional[dict] = None,
        optim_alpha: Optional[float] = None,
        optim_eps: Optional[float] = None,
        grad_norm_clipping: Optional[float] = None,
        grad_clip: Optional[float] = None,
        **kwargs,
    ) -> "QMixConfig":
        """Sets the training related configuration.
        Args:
            mixer: Mixing network. Either "qmix", "vdn", or None.
            mixing_embed_dim: Size of the mixing network embedding.
            double_q: Whether to use Double_Q learning.
            target_network_update_freq: Update the target network every
                `target_network_update_freq` sample steps.
            replay_buffer_config:
            optim_alpha: RMSProp alpha.
            optim_eps: RMSProp epsilon.
            grad_clip: If not None, clip gradients during optimization at
                this value.
            grad_norm_clipping: Depcrecated in favor of grad_clip
        Returns:
            This updated AlgorithmConfig object.
        """
        # Pass kwargs onto super's `training()` method.
        super().training(**kwargs)

        if grad_norm_clipping is not None:
            deprecation_warning(
                old="grad_norm_clipping",
                new="grad_clip",
                help="Parameter `grad_norm_clipping` has been "
                "deprecated in favor of grad_clip in QMix. "
                "This is now the same parameter as in other "
                "algorithms. `grad_clip` will be overwritten by "
                "`grad_norm_clipping={}`".format(grad_norm_clipping),
                error=False,
            )
            grad_clip = grad_norm_clipping

        if mixer is not None:
            self.mixer = mixer
        if mixing_embed_dim is not None:
            self.mixing_embed_dim = mixing_embed_dim
        if double_q is not None:
            self.double_q = double_q
        if target_network_update_freq is not None:
            self.target_network_update_freq = target_network_update_freq
        if replay_buffer_config is not None:
            self.replay_buffer_config = replay_buffer_config
        if optim_alpha is not None:
            self.optim_alpha = optim_alpha
        if optim_eps is not None:
            self.optim_eps = optim_eps
        if grad_clip is not None:
            self.grad_clip = grad_clip

        return self


class QMix(SimpleQ):
    @classmethod
    @override(SimpleQ)
    def get_default_config(cls) -> AlgorithmConfigDict:
        return QMixConfig().to_dict()

    @override(SimpleQ)
    def validate_config(self, config: AlgorithmConfigDict) -> None:
        # Call super's validation method.
        super().validate_config(config)

        if config["framework"] != "torch":
            raise ValueError("Only `framework=torch` supported so far for QMix!")

    @override(SimpleQ)
    def get_default_policy_class(self, config: AlgorithmConfigDict) -> Type[Policy]:
        return QMixTorchPolicy

    @override(SimpleQ)
    def training_step(self) -> ResultDict:
        """QMIX training iteration function.
        - Sample n MultiAgentBatches from n workers synchronously.
        - Store new samples in the replay buffer.
        - Sample one training MultiAgentBatch from the replay buffer.
        - Learn on the training batch.
        - Update the target network every `target_network_update_freq` sample steps.
        - Return all collected training metrics for the iteration.
        Returns:
            The results dict from executing the training iteration.
        """
        # Sample n batches from n workers.
        new_sample_batches = synchronous_parallel_sample(
            worker_set=self.workers, concat=False
        )

        for batch in new_sample_batches:
            # Update counters.
            self._counters[NUM_ENV_STEPS_SAMPLED] += batch.env_steps()
            self._counters[NUM_AGENT_STEPS_SAMPLED] += batch.agent_steps()
            # Store new samples in the replay buffer.
            self.local_replay_buffer.add(batch)

        # Update target network every `target_network_update_freq` sample steps.
        cur_ts = self._counters[
            NUM_AGENT_STEPS_SAMPLED if self._by_agent_steps else NUM_ENV_STEPS_SAMPLED
        ]

        train_results = {}

        if cur_ts > self.config["num_steps_sampled_before_learning_starts"]:
            # Sample n batches from replay buffer until the total number of timesteps
            # reaches `train_batch_size`.
            train_batch = sample_min_n_steps_from_buffer(
                replay_buffer=self.local_replay_buffer,
                min_steps=self.config["train_batch_size"],
                count_by_agent_steps=self._by_agent_steps,
            )

            # Learn on the training batch.
            # Use simple optimizer (only for multi-agent or tf-eager; all other
            # cases should use the multi-GPU optimizer, even if only using 1 GPU)
            if self.config.get("simple_optimizer") is True:
                train_results = train_one_step(self, train_batch)
            else:
                train_results = multi_gpu_train_one_step(self, train_batch)

            # Update target network every `target_network_update_freq` sample steps.
            last_update = self._counters[LAST_TARGET_UPDATE_TS]
            if cur_ts - last_update >= self.config["target_network_update_freq"]:
                to_update = self.workers.local_worker().get_policies_to_train()
                self.workers.local_worker().foreach_policy_to_train(
                    lambda p, pid: pid in to_update and p.update_target()
                )
                self._counters[NUM_TARGET_UPDATES] += 1
                self._counters[LAST_TARGET_UPDATE_TS] = cur_ts

            update_priorities_in_replay_buffer(
                self.local_replay_buffer, self.config, train_batch, train_results
            )

            # Update weights and global_vars - after learning on the local worker -
            # on all remote workers.
            global_vars = {
                "timestep": self._counters[NUM_ENV_STEPS_SAMPLED],
            }
            # Update remote workers' weights and global vars after learning on local
            # worker.
            with self._timers[SYNCH_WORKER_WEIGHTS_TIMER]:
                self.workers.sync_weights(global_vars=global_vars)

        # Return all collected metrics for the iteration.
        return train_results

In [50]:
from gym.spaces import Discrete, MultiDiscrete,Box, Dict, MultiBinary,Tuple
from ray.tune.registry import get_trainable_cls

import argparse
import logging
import os

import ray
from ray import air, tune
from ray.tune import register_env
from ray.rllib.env.multi_agent_env import ENV_STATE
from ray.rllib.examples.env.two_step_game import TwoStepGame
from ray.rllib.policy.policy import PolicySpec
from ray.rllib.utils.test_utils import check_learning_achieved
from ray.tune.registry import get_trainable_cls

In [56]:
parser = argparse.ArgumentParser()

"""
                {
                    "agent0": Discrete(1),#agent 0 will be sour external agent/motor neuron
                    "agent1": Discrete(1),#Discrete(1), #excitatory neurons
                    "agent2": gym.spaces.Box(low=0, high=1.0, shape=(10,))#Discrete(1)#inhibitory neurons
                }


"""
if __name__ == "__main__":
    args = parser.parse_args()

    #ray.init(num_cpus=args.num_cpus or None, local_mode=args.local_mode)

    grouping = {
        "group_1": [0, 1,2],
    }
    obs_space = gym.spaces.Tuple(
        [
            gym.spaces.Dict(
                {
                    "obs": MultiDiscrete([2, 2, 2, 3]),
                    ENV_STATE: MultiDiscrete([2, 2, 2]),
                }
            ),
            gym.spaces.Dict(
                {
                    "obs": MultiDiscrete([2, 2, 2, 3]),
                    ENV_STATE: MultiDiscrete([2, 2, 2]),
                }
            ),
        ]
    )
    act_space = gym.spaces.Tuple(
        [
            gym.spaces.Dict(
                {
                    "agent0": Discrete(1),#agent 0 will be sour external agent/motor neuron
                }
            ),
            
        ]
    )
    register_env(
        "grouped_twostep",
        lambda config: TwoStepGame(config).with_agent_groups(
            grouping, obs_space=obs_space, act_space=act_space
        ),
    )

    config = (
        get_trainable_cls(args.run)
        .get_default_config()
        .environment(TwoStepGame)
        .framework(args.framework)
        # Use GPUs iff `RLLIB_NUM_GPUS` env var set to > 0.
        .resources(num_gpus=int(os.environ.get("RLLIB_NUM_GPUS", "0")))
    )
    (
    config.framework("torch")
        .training(mixer=args.mixer, train_batch_size=32)
        .rollouts(num_rollout_workers=0, rollout_fragment_length=4)
        .exploration(
            exploration_config={
                    "final_epsilon": 0.0,
            }
        )
        .environment(
            env="grouped_twostep",
            env_config={
                "separate_state_space": True,
                "one_hot_state_encoding": True,
            },
        )
    )

    stop = {
        "episode_reward_mean": args.stop_reward,
        "timesteps_total": args.stop_timesteps,
        "training_iteration": args.stop_iters,
    }

    results = tune.Tuner(
        args.run,
        run_config=air.RunConfig(stop=stop, verbose=2),
        param_space=config,
    ).fit()

    if args.as_test:
        check_learning_achieved(results, args.stop_reward)

    ray.shutdown()

AttributeError: 'Namespace' object has no attribute 'run'

In [16]:
from ray.rllib.algorithms.qmix import QMixConfig
from ray import air
from ray import tune
config = QMixConfig()
# Print out some default values.
print(config.optim_alpha)  
# Update the config object.
config.training(  
    lr=tune.grid_search([0.001, 0.0001]), optim_alpha=0.97
)
# Set the config object's env.
#print(config.optim_alpha) 
config.environment(env=TwoStepGame)  
# Use to_dict() to get the old-style python config dict
# when running with tune.
print(config.optim_alpha) 

#only the tune.tuner code is breaking. Everything up to this point is compiling

tune.Tuner(  
    "QMix",
    run_config=air.RunConfig(stop={"episode_reward_mean": 200}),
    param_space=config.to_dict(),
).fit()

0.99
0.97


TuneError: The Ray Tune run failed. Please inspect the previous error messages for a cause. After fixing the issue, you can restart the run from scratch or continue this run. To continue this run, you can use `tuner = Tuner.restore("C:\Users\subar\ray_results\QMix")`.

In [17]:
from ray.rllib.examples.env.two_step_game import TwoStepGame
from ray.rllib.algorithms.qmix import QMixConfig
config = QMixConfig()  # doctest: +SKIP
print("adf")
config = config.training(gamma=0.9, lr=0.01)#, kl_coeff=0.3)  # doctest: +SKIP

config = config.resources(num_gpus=0)  # doctest: +SKIP
config = config.rollouts(num_rollout_workers=12)  # doctest: +SKIP
print(config.to_dict())  # doctest: +SKIP

# Build an Algorithm object from the config and run 1 training iteration.
algo = config.build(env=TwoStepGame)  # doctest: +SKIP

algo.train()  # doctest: +SKIP

adf
{'extra_python_environs_for_driver': {}, 'extra_python_environs_for_worker': {}, 'num_gpus': 0, 'num_cpus_per_worker': 1, 'num_gpus_per_worker': 0, '_fake_gpus': False, 'custom_resources_per_worker': {}, 'placement_strategy': 'PACK', 'eager_tracing': False, 'eager_max_retraces': 20, 'tf_session_args': {'intra_op_parallelism_threads': 2, 'inter_op_parallelism_threads': 2, 'gpu_options': {'allow_growth': True}, 'log_device_placement': False, 'device_count': {'CPU': 1}, 'allow_soft_placement': True}, 'local_tf_session_args': {'intra_op_parallelism_threads': 8, 'inter_op_parallelism_threads': 8}, 'env': None, 'env_config': {}, 'observation_space': None, 'action_space': None, 'env_task_fn': None, 'render_env': False, 'clip_rewards': None, 'normalize_actions': True, 'clip_actions': False, 'disable_env_checking': False, 'num_envs_per_worker': 1, 'sample_collector': <class 'ray.rllib.evaluation.collectors.simple_list_collector.SimpleListCollector'>, 'sample_async': False, 'enable_connector

2023-01-30 21:22:08,776	INFO worker.py:1529 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
(pid=17056) C:\Users\subar\anaconda3\envs\d\lib\site-packages\tensorflow\core\framework\tensor_shape_pb2.py:18: DeprecationWarning: Call to deprecated create function FileDescriptor(). Note: Create unlinked descriptors is going to go away. Please use get/find descriptors from generated code or query the descriptor_pool.
(pid=17056)   DESCRIPTOR = _descriptor.FileDescriptor(
(pid=17056) C:\Users\subar\anaconda3\envs\d\lib\site-packages\tensorflow\core\framework\tensor_shape_pb2.py:36: DeprecationWarning: Call to deprecated create function FieldDescriptor(). Note: Create unlinked descriptors is going to go away. Please use get/find descriptors from generated code or query the descriptor_pool.
(pid=17056)   _descriptor.FieldDescriptor(
(pid=17056) C:\Users\subar\anaconda3\envs\d\lib\site-packages\tensorflow\core\framework\tensor_shape_pb2.py:29: DeprecationWarning: Call to de

ValueError: Obs space must be a Tuple, got Box(-1.0, 1.0, (6,), float32). Use MultiAgentEnv.with_agent_groups() to group related agents for QMix.

In [3]:
import tensorflow as tf
import numpy as np
import sys

#sys.path.append("../src")

import os
#os.environ['EEG_FMRI_DATASETS']="/home/david/eeg_to_fmri/datasets"
#os.environ['EEG_FMRI']="/home/david/eeg_to_fmri"

import eeg_to_fmri
from eeg_to_fmri.utils import tf_config

#dataset="01"
tf_config.set_seed(seed=2)
#tf_config.setup_tensorflow(device="GPU", memory_limit=1500, run_eagerly=True)

from eeg_to_fmri.models.synthesizers import EEG_to_fMRI
from eeg_to_fmri.data import preprocess_data, eeg_utils, data_utils
from eeg_to_fmri.learning import train, losses
from eeg_to_fmri import metrics
from eeg_to_fmri.utils import viz_utils
from eeg_to_fmri.models.synthesizers import na_specification_eeg
from eeg_to_fmri.models.fmri_ae import na_specification_fmri



In [ ]:
with tf.device('/CPU:0'):
    model = EEG_to_fMRI(latent_dimension, eeg_train.shape[1:], na_specification_eeg, n_channels,
                        weight_decay=weight_decay, skip_connections=True, batch_norm=True, fourier_features=True,
                        random_fourier=True, topographical_attention=True, conditional_attention_style=True,
                        conditional_attention_style_prior=False, local=True, seed=None, 
                        fmri_args = (latent_dimension, fmri_train.shape[1:], kernel_size, stride_size, n_channels, 
                        max_pool, batch_norm, weight_decay, skip_connections,
                        n_stacks, True, False, outfilter, dropout, None, False, na_specification_fmri))
    model.build(eeg_train.shape, fmri_train.shape)
    optimizer = tf.keras.optimizers.Adam(learning_rate)
    loss_fn = losses.mae_cosine
    train_set = tf.data.Dataset.from_tensor_slices((eeg_train, fmri_train)).batch(batch_size)
    test_set= tf.data.Dataset.from_tensor_slices((eeg_test, fmri_test)).batch(1)

In [23]:
from surfer import Brain

sub = 'fsaverage'
hemi = 'lh'
surf = 'inflated'

brain = Brain(sub, hemi, surf)

brain.animate(['l', 'c'])

# control number of steps
brain.animate(['l', 'm'], n_steps=30)

# any path you can think of
brain.animate(['l', 'c', 'm', 'r', 'c', 'r', 'l'], n_steps=45)

# full turns
brain.animate(["m"] * 3)

# movies
brain.animate(['l', 'l'], n_steps=10, fname='simple_animation.avi')

# however, rotating out of the axial plane is not allowed
try:
    brain.animate(['l', 'd'])
except ValueError as e:
    print(e)


ValueError: The subjects directory has to be specified using the subjects_dir parameter or the SUBJECTS_DIR environment variable.